In [1]:
import csv

In [2]:
%%time
with open('checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

Wall time: 3.81 s


In [3]:
%%time
with open('checkins.csv', 'w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

Wall time: 2.2 s


In [4]:
import numpy as np
import pandas as pd

In [5]:
%%time
data = pd.read_csv('checkins.csv', header=0)

Wall time: 1.16 s


In [6]:
len(data) == 396634

True

In [7]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [8]:
import sklearn.cluster as cluster

X = data.values[0:100000, 3:5]
cluster1 = cluster.MeanShift(bandwidth=0.1)

In [9]:
cluster1.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [10]:
labels = cluster1.labels_
cluster_centers = cluster1.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

3230

In [11]:
cluster_centers[1]

array([  33.44943805, -112.00213969])

In [12]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1

In [13]:
count = 0
for key in d.keys():
    if d[key] > 15:
        count += 1
print(count)

592


In [14]:
clusters_select = np.ndarray(shape=(count,2))

In [15]:
clusters_select

array([[  9.08817280e-312,   9.08817227e-312],
       [  8.01282439e-315,   6.23510845e-321],
       [  9.09057200e-312,   8.01282439e-315],
       ..., 
       [  0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000]])

In [16]:
i = 0
j = 0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j += 1
    i += 1

In [17]:
len(clusters_select)

592

In [18]:
clusters_select[0]

array([ 40.7177164 , -73.99183542])

In [19]:
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])

In [20]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [21]:
distance(offices[0], clusters_select[123])

20.892897123756782

In [22]:
answer_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(xx, clusters_select[i]) for xx in offices]
    if min_dist == 0:
        min_dist = min(distances)
        answer_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            answer_index = i
    i += 1

In [23]:
clusters_select[answer_index]

array([ -33.86063043,  151.20477593])

In [24]:
def write_answer(center):
    with open("answer.txt", "w") as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [25]:
write_answer(clusters_select[answer_index])